In [1]:
%matplotlib inline 
# 魔法命令，使用后画图不用show了

import numpy as np
import matplotlib.pyplot as plt
import re# 引入正则

import warnings
warnings.filterwarnings("ignore")

# 1.解压词向量并加载

## 1.1解压词向量

In [2]:
import bz2# 用来解压文件

In [3]:
with open("./embeddings/sgns.weibo.bigram", 'wb') as new_file, open("./embeddings/sgns.weibo.bigram.bz2", 'rb') as file:
    decompressor = bz2.BZ2Decompressor()
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(decompressor.decompress(data))

## 1.2加载词向量

In [4]:
from gensim.models import KeyedVectors# gensim用来加载预训练词向量

In [5]:
cn_model = KeyedVectors.load_word2vec_format('./embeddings/sgns.weibo.bigram', 
                                             binary=False,
                                             unicode_errors="ignore")

# 2.语料预处理

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 2.1读取原始文本
* weibo：DataFrame存储的博文及其对应标签
* content：list存储的原始文本字符串
* label：标签，1为非谣言

In [7]:
import pandas as pd

In [8]:
weibo = pd.read_csv('./data/all_data.txt',sep='\t', names=['is_not_rumor','content'],encoding='utf-8')
weibo = weibo.dropna()#删除缺失值
weibo.head()

,is_not_rumor,content
0,0,2013年3月5日，《明报》做了题目为《如果有来生，你愿不愿意再做中国人？》的投票调查，截止...
1,1,各位同学，有认识住在华昌路靠近中兴路的海伦新苑的朋友吗？转给他们看一下好伐……帮在下留意一下...
2,1,#北京曝光# @倍儿小爽 ：今天回家路上接到老妈电话，老妈冒雨给我送饭，拿到饭我让老妈打车回...
3,0,一位可怜的94岁老奶奶，老伴去世，儿子伤寒死了，两个孙子在外面打工，她每天依靠捡垃圾为生，每...
4,1,有人好奇黑人妹子是如何化妆的么？LZ带图详解 [哈哈] 太搞笑了，一直以为他们是不化妆的～～（转）


In [9]:
weibo.shape

(3387, 2)

In [10]:
#将DataFrame中的Series转换为list
content = weibo.content.values.tolist()
label=weibo.is_not_rumor.values.tolist()

In [11]:
print (content[3:5])

['一位可怜的94岁老奶奶，老伴去世，儿子伤寒死了，两个孙子在外面打工，她每天依靠捡垃圾为生，每天捡垃圾捡到凌晨2点，也只能转5、6元。奶奶双腿已经裂开，因为没钱，一直没有医治。求扩散~~每转一条微博，腾讯公益就像老奶奶捐出1毛钱，多转几次吧，不会脏了你微博，对么？', '有人好奇黑人妹子是如何化妆的么？LZ带图详解 [哈哈] 太搞笑了，一直以为他们是不化妆的～～（转）']


## 2.2进行分词和tokenize

https://github.com/lancopku/PKUSeg-python

对每一条微博文本text，
1. 去掉每个样本的标点符号；
2. 用pkuseg分词，得到存放分词结果的cut_list；
3. 去掉cut_list中的停用词得到cut_list_clean；
3. 将分词结果cut_list_clean索引化（使用北京师范大学中文信息处理研究所与中国人民大学 DBIIR 实验室的研究者开源的"chinese-word-vectors"），这样每一例评价的文本变成一段索引数字，对应着预训练词向量模型中的词。

将每个text的结果存到train_tokens中。

In [12]:
import pkuseg

In [14]:
#导入停用词
stopwords=pd.read_csv("./stopwords/stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
stopwords = stopwords.stopword.values.tolist()#转为list形式

In [15]:
seg = pkuseg.pkuseg(model_name='web')  # 程序会自动下载所对应的细领域模型

Downloading: "https://github.com/lancopku/pkuseg-python/releases/download/v0.0.16/web.zip" to C:\Users\dell/.pkuseg\web.zip
100%|█████████████████████████| 17478354/17478354 [00:12<00:00, 1432747.33it/s]


In [17]:
train_tokens = []
for text in content:
    # 去掉标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # pkuseg分词
    cut_list = seg.cut(text)

    #去除停用词
    cut_list_clean=[]
    for word in cut_list:
        if word in stopwords:
            continue
        cut_list_clean.append(word)
    
    #索引化
    for i, word in enumerate(cut_list_clean): # enumerate()
        try:
            # 将词转换为索引index
            cut_list_clean[i] = cn_model.vocab[word].index
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list_clean[i] = 0
    train_tokens.append(cut_list_clean)

## 2.3索引长度标准化

因为每段评语的长度是不一样的，如果单纯取最长的一个评语，并把其他评填充成同样的长度，这样十分浪费计算资源，所以取一个折衷的长度。

In [18]:
# 获得所有tokens的长度
num_tokens = [len(tokens) for tokens in train_tokens]
num_tokens = np.array(num_tokens)
# 取tokens平均值并加上两个tokens的标准差，
# 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

58

### padding（填充）和truncating（修剪）

我们把文本转换为tokens（索引）之后，每一串索引的长度并不相等，所以为了方便模型的训练我们需要把索引的长度标准化，上面我们选择了max_tokens个可以涵盖95%训练样本的长度，接下来我们进行padding和truncating，我们一般采用'pre'的方法，这会在文本索引的前面填充0，因为根据一些研究资料中的实践，如果在文本索引后面填充0的话，会对模型造成一些不良影响。 

进行padding和truncating， 输入的train_tokens是一个list
返回的train_pad是一个numpy array

In [19]:
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')

## 2.4准备Embedding Matrix

* 作为模型的输入，需要准备一个维度为 (𝑛𝑢𝑚𝑤𝑜𝑟𝑑𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚) 的embedding矩阵，num words代表使用的词汇的数量。


* 不进行词向量的训练，而是使用预训练的词向量——北京师范大学中文信息处理研究所与中国人民大学 DBIIR 实验室的研究者开源的"chinese-word-vectors"；https://github.com/Embedding/Chinese-Word-Vectors ；emdedding dimension在现在使用的预训练词向量模型中是300，每一个词汇都用一个长度为300的向量表示。


* 注意只选择使用前50k个使用频率最高的词，在这个预训练词向量模型中，一共有260万词汇量，如果全部使用在分类问题上会很浪费计算资源，因为训练样本很小，如果有更多的训练样本时，在分类问题上可以考虑减少使用的词汇量。

In [20]:
num_words = 50000
embedding_dim=300
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50000 * 300
for i in range(num_words):
    embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]#前50000个index对应的词的词向量
embedding_matrix = embedding_matrix.astype('float32')
# 检查index是否对应，
# 输出300意义为长度为300的embedding向量一一对应
np.sum(cn_model[cn_model.index2word[333]] == embedding_matrix[333] )

300

In [22]:
# 超出五万个词向量的词用0代替
train_pad[train_pad>=num_words ] = 0

# 准备target向量，前2000样本为1，后2000为0
train_target = np.array(label)

# 3.训练语料

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 3.1划分训练集和测试集

In [24]:
# 进行训练和测试样本的分割
from sklearn.model_selection import train_test_split
# 90%的样本用来训练，剩余10%用来测试
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=12)

## 3.2搭建网络结构

* 用keras搭建LSTM模型，模型的第一层是Embedding层，只有当我们把tokens索引转换为词向量矩阵之后，才可以用神经网络对文本进行处理。 keras提供了Embedding接口，避免了繁琐的稀疏矩阵操作。
* 在Embedding层我们输入的矩阵为：(𝑏𝑎𝑡𝑐ℎ𝑠𝑖𝑧𝑒,𝑚𝑎𝑥𝑡𝑜𝑘𝑒𝑛𝑠)输出矩阵为:(𝑏𝑎𝑡𝑐ℎ𝑠𝑖𝑧𝑒,𝑚𝑎𝑥𝑡𝑜𝑘𝑒𝑛𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚)。
* 使用预训练的词向量，将trainable设为False，即不可训练。

In [25]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Bidirectional(LSTM(units=32, return_sequences=False)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer=Adam(lr=1e-3)

==================model1==================

In [37]:
model1 = Sequential()
model1.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model1.add(Bidirectional(GRU(32)))
model1.add(Dense(6, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
optimizer=Adam(lr=1e-3)

## 3.3模型配置

### 模型保存（断点续训）、early stoping、学习率

In [27]:
import os

In [28]:
# 建立一个权重的存储点
checkpoint_save_path="./checkpoint/rumor_LSTM.ckpt"
if os.path.exists(checkpoint_save_path+'.index'):
    print('----------load the model----------')
    model.load_weights(checkpoint_save_path)

In [29]:
#保存参数和模型
checkpoint = ModelCheckpoint(filepath=checkpoint_save_path, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [30]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
# 定义callback函数
callbacks = [
    earlystopping, 
#    checkpoint,
    lr_reduction
]

In [31]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

========================model1===================

In [38]:
# 建立一个权重的存储点
checkpoint_save_path1="./checkpoint/rumor_GRU.ckpt"
if os.path.exists(checkpoint_save_path1+'.index'):
    print('----------load the model----------')
    model1.load_weights(checkpoint_save_path1)

In [39]:
#保存参数和模型
checkpoint1 = ModelCheckpoint(filepath=checkpoint_save_path1, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [40]:
model1.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 3.4训练模型

In [32]:
model.fit(X_train, y_train,validation_split=0.1,epochs=20,batch_size=128,callbacks=callbacks)

Train on 2743 samples, validate on 305 samples
Epoch 1/20
2743/2743 [==============================] - 69s 25ms/sample - loss: 0.6390 - accuracy: 0.6482 - val_loss: 0.4600 - val_accuracy: 0.8295
Epoch 2/20
2688/2743 [============================>.] - ETA: 0s - loss: 0.4470 - accuracy: 0.8039
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2743/2743 [==============================] - 14s 5ms/sample - loss: 0.4463 - accuracy: 0.8046 - val_loss: 0.4779 - val_accuracy: 0.8000
Epoch 3/20
2743/2743 [==============================] - 11s 4ms/sample - loss: 0.4171 - accuracy: 0.8181 - val_loss: 0.3556 - val_accuracy: 0.8426
Epoch 4/20
2743/2743 [==============================] - 11s 4ms/sample - loss: 0.3767 - accuracy: 0.8400 - val_loss: 0.3397 - val_accuracy: 0.8623
Epoch 5/20
2743/2743 [==============================] - 10s 4ms/sample - loss: 0.3627 - accuracy: 0.8498 - val_loss: 0.3333 - val_accuracy: 0.8590
Epoch 6/20
2743/2743 [===========================

======================model1===========================

In [41]:
model1.fit(X_train, y_train,validation_split=0.1,epochs=20,batch_size=128,callbacks=callbacks)

Train on 2743 samples, validate on 305 samples
Epoch 1/20
2743/2743 [==============================] - 29s 11ms/sample - loss: 0.7042 - accuracy: 0.5035 - val_loss: 0.6928 - val_accuracy: 0.5279
Epoch 2/20
2743/2743 [==============================] - 7s 2ms/sample - loss: 0.6920 - accuracy: 0.5472 - val_loss: 0.6910 - val_accuracy: 0.5279
Epoch 3/20
2743/2743 [==============================] - 4s 1ms/sample - loss: 0.6785 - accuracy: 0.5472 - val_loss: 0.6643 - val_accuracy: 0.5279
Epoch 4/20
2743/2743 [==============================] - 4s 1ms/sample - loss: 0.6348 - accuracy: 0.5472 - val_loss: 0.6109 - val_accuracy: 0.5279
Epoch 5/20
2743/2743 [==============================] - 4s 2ms/sample - loss: 0.5728 - accuracy: 0.5633 - val_loss: 0.5394 - val_accuracy: 0.7475
Epoch 6/20
2743/2743 [==============================] - 5s 2ms/sample - loss: 0.4963 - accuracy: 0.8064 - val_loss: 0.4369 - val_accuracy: 0.8459
Epoch 7/20
2743/2743 [==============================] - 4s 1ms/sample - los

## 3.5应用于测试集
首先对测试样本进行预测，得到了还算满意的准确度。之后我们定义一个预测函数，来预测输入的文本的极性，可见模型对于否定句和一些简单的逻辑结构都可以进行准确的判断。

In [33]:
result = model.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

339/339 [==============================] - 2s 5ms/sample - loss: 0.3997 - accuracy: 0.8289
Accuracy:82.89%


======================model1===========================

In [42]:
result = model1.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

339/339 [==============================] - 1s 3ms/sample - loss: 0.3591 - accuracy: 0.8614
Accuracy:86.14%


## 3.6实例展示

In [45]:
def predict_rumor_LSTM(text,label):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = seg.cut(text)
    #去除停用词
    cut_clean=[]
    for word in cut:
        if word in stopwords:
            continue
        cut_clean.append(word)
    # tokenize
    for i, word in enumerate(cut_clean):
        try:
            cut_clean[i] = cn_model.vocab[word].index
            if cut_clean[i] >= 50000:
                cut_clean[i] = 0
        except KeyError:
            cut_clean[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_clean], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    dic={0:'谣言',1:'非谣言'}
    result = model.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('真实是'+dic[label],'预测是非谣言','output=%.2f'%coef)
    else:
        print('真实是'+dic[label],'预测是谣言','output=%.2f'%coef)
    print('---------------------------------------------')

In [46]:
test_list = [
    '兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！',
    '重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下',
    '@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻',
]
test_label=[0,0,1]
for i in range(len(test_list)):
    predict_rumor_LSTM(test_list[i],test_label[i])

兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！
真实是谣言 预测是谣言 output=0.10
---------------------------------------------
重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下
真实是谣言 预测是谣言 output=0.10
---------------------------------------------
@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻
真实是非谣言 预测是非谣言 output=0.71
---------------------------------------------


=====================model1=========================

In [47]:
def predict_rumor_GRU(text,label):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = seg.cut(text)
    #去除停用词
    cut_clean=[]
    for word in cut:
        if word in stopwords:
            continue
        cut_clean.append(word)
    # tokenize
    for i, word in enumerate(cut_clean):
        try:
            cut_clean[i] = cn_model.vocab[word].index
            if cut_clean[i] >= 50000:
                cut_clean[i] = 0
        except KeyError:
            cut_clean[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_clean], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    dic={0:'谣言',1:'非谣言'}
    result = model1.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('真实是'+dic[label],'预测是非谣言','output=%.2f'%coef)
    else:
        print('真实是'+dic[label],'预测是谣言','output=%.2f'%coef)
    print('---------------------------------------------')

In [49]:
test_list = [
    '兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！',
    '重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下',
    '@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻',
]
test_label=[0,0,1]
for i in range(len(test_list)):
    predict_rumor_GRU(test_list[i],test_label[i])

兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！
真实是谣言 预测是谣言 output=0.10
---------------------------------------------
重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下
真实是谣言 预测是谣言 output=0.11
---------------------------------------------
@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻
真实是非谣言 预测是非谣言 output=0.61
---------------------------------------------
